In [6]:
import lightgbm as lgb  # standard alias
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import norm
from sklearn.model_selection import train_test_split
import math
from catboost import CatBoostRegressor
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance


train = pd.read_csv('../feature_engineering/train_set.csv')
test = pd.read_csv('../feature_engineering/test_set.csv')

x_train, x_test, y_train, y_test = train_test_split(train, train['revenue'], test_size=0.2, random_state=42)

# Save store ids, then remove this column
store_ids = x_test['store_id'].to_numpy()
ids = pd.DataFrame(store_ids)
ids.rename(columns={0 :'id'}, inplace=True )

x_test = x_test.drop('store_id', axis=1)


#X_train = X_train.drop('grunnkrets_id', axis=1)
#X_test = X_test.drop('grunnkrets_id', axis=1)
x_test = x_test.drop('mall_dummy', axis=1)

x_train = x_train.fillna(0)
y_train = y_train.fillna(0)
x_test = x_test.fillna(0)
x_train = 0
y_train = 0
x_test = 0
y_test = 0

In [7]:
def catboost():
    model = CatBoostRegressor(iterations=1000,
                            learning_rate=0.005,
                            depth=12,
                            loss_function= 'RMSE')
    # Fit model
    model.fit(x_train, y_train)
    # Get predictions
    preds = model.predict(x_test)
    preds = np.expm1(preds)

    #generate csv file of id and prediction
    predicted = pd.DataFrame(preds, columns = ['predicted'])
    output = pd.concat([ids,predicted],axis=1)
    output.to_csv('cat.csv',index=False)

In [8]:
def lgbm():
    #Hyper parameters
    hyper_params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': ['l1','l2'],
        'learning_rate': 0.005,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.7,
        'bagging_freq': 10,
        'verbose': 0,
        "max_depth": 8,
        "num_leaves": 128,  
        "max_bin": 512,
        "num_iterations": 1000
    }

    #fit the model takes some time
    gbm = lgb.LGBMRegressor(**hyper_params)
    gbm.fit(x_train, y_train)
    print("fit")
    #predict
    y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration_)
    y_pred = np.expm1(y_pred)

    #generate csv file of id and prediction
    predicted = pd.DataFrame(y_pred, columns = ['predicted'])
    output = pd.concat([ids,predicted],axis=1)
    output.to_csv('lgbm.csv',index=False)